# Финальный проект по курсу "Сбор данных с Web-scraping и API для социально-научных исследований", 2023-2024 гг.

#### *Проект выполнили студентки 4 курса ОП "Политология" <br> Бойцова Анастасия, Дячок Дарья*

Проект заключается в сборе данных музыкальных рейтингов за период с 2020 года по сегодняшний день и составлении базы данных с текстами самых популярных композиций последних лет для последующего их анализа. Цель проекта состоит в ответе на следующий вопрос: 
> "Как изменились музыкальные предпочтения российского слушателя на фоне социально-политических событий последних лет?" 

Данные парсились при помощи методов <u>Selenium</u> c сайтов [Apple Music](https://music.apple.com/ru/browse) и [Genius](https://genius.com/): Подробнее о цели проекта и соотношении с социальными науками можно прочитать [по ссылке](https://docs.google.com/document/d/16dVCcUWZtEaOPDfabYvcyMh4dVktnthblbVTIEl-9-c/edit?usp=sharing).


***

## Часть 1. Сбор данных

### Шаг 1. 

Импортируем необходимые для работы библиотеки.

In [1]:
import pandas as pd # для работы с датафреймами
import time # модуль времени
import re # для функция re.sub

from datetime import datetime # для создания объектов типа дата-время

# модули для работы с веб-драйвером
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException


### Шаг 2. Сбор данных с Apple Music

Запускаем сущность веб-драйвер. Посылаем запрос к сайту [Apple Music](https://music.apple.com/ru/browse).

In [95]:
options = Options() 
options.add_argument("start-maximized") # передаем аргумент в options, чтобы окно открывалось на полный экран 
wb = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

wait = WebDriverWait(wb, 10) # задаем wait: мы будем ждать до 10 секунд или пока не найдем элемент

In [96]:
url_apple = "https://music.apple.com/ru/browse"
wb.get(url_apple)

Находим кнопку поиска и отправляем в нее запрос *Топ 100 Россия*. <br><br> Собираем ссылки всех рейтинговых плейлистов, по которым потом будет проходить наш вебдрайвер: по условиям поставленной нами же задачи нас интересуют только рейтинги за последние четыре года, соответственно, мы просим вебдрайвер собрать только года после 2020. За 2023 год принимаем рейтинг "Top 100: Russia", в который входят наиболее популярные (с кумулятивной точки зрения) треки с начала 2023 года и по сегодняшний день. <br><br> Определяем функцию <font color='blue'>process_data</font>, при помощи которой будем собирать данные о треках с четырех страниц плейлистов.

In [97]:
top1_selection_button = wait.until(EC.element_to_be_clickable(
    (By.XPATH, '/html/body/div/div[2]/nav/div[1]/div[2]/div/div[1]/form/input')))
top1_selection_button.send_keys('Топ 100 Россия' + '\n') # в конце добавляем \n, чтобы сразу запустить поиск

wb.implicitly_wait(3)
playlists = wb.find_elements(By.XPATH, '//a[@data-testid="click-action" and (contains(@aria-label, "Топ-100 России: 202") or contains(@aria-label, "Top 100: Russia"))]')
playlist_urls = [link.get_attribute('href') for link in playlists]

In [98]:
def process_data(url):
    
    '''Функция process_data() принимает на вход url-ссылку, обращается к элементам на сайте при помощи библиотеки
    Selenium и возвращает pandas.DataFrame с информацией о каждом элементе музыкального рейтинга.
    
    Функция создана для парсинга сайта https://music.apple.com/ru/browse и не предназначена для использования 
    на других ресурсах.
    
    Arguments:
        > url - str - ccылка на плейлист на платформе Apple Music;
    Returns:
        > pandas.DataFrame - с информацией об элементах музыкального рейтинга:
            : Название трека - str - название песни;
            : Номер в рейтинге - int - позиция песни в рамках данного рейтинга;
            : Исполнитель - str - исполнител(и) песни;
            : Альбом - str - альбом, в который входит песня;
            : Длительность - str - длительность песни в формате {м:сс}.
            : Ненормативный контент - bool - присутствие ненормативного контента в песне = 1, 
                                             отсутствие ненормативного контента в песне = 0
            : Год вхождения в топ - int - год рейтинга, в который вошля песня
            : Ссылка на прослушивание - str - ссылка на прослушивание песни на Apple Music
    Example:
        >>> example = get_last_posts('https://music.apple.com/ru/browse')
        >>> example
    
    '''
    
    
    # 1. Получаем элементы
    
    elements = wb.find_elements(By.CLASS_NAME, 'songs-list-row')
    E = wb.find_elements(By.XPATH, '//div[@data-testid="track-list-item"]') # excplicit
    year = wb.find_element(By.TAG_NAME, 'h1').text.split()[-1] # year
    link_elements = wb.find_elements(By.CSS_SELECTOR, 'a[data-testid="track-seo-link"]') # link
    
    
    # 2. Получаем значения
    
    # заменяем точку, которой разделены исполнители, когда их несколько
    # т.к. иначе возникают проблемы на следующем этапе (отправка запроса на сайте Genius)
    main_values = [element.text.replace('•', '&') for element in elements] 
    explicit = []
    for one in E:
        one = one.get_attribute("aria-label").split(',')[0]
        explicit.append((lambda x: 1 if x == 'Ненормативный контент' else 0)(one))
    year = int((lambda x: '2023' if x == 'Russia' else x)(year))
    link = [element.get_attribute('href') for element in link_elements]
    
    # 3. Разделяем значения и формируем датафрейм
    
    values_list = [item.split('\n') for item in main_values]
    df = pd.DataFrame(values_list, columns = ['Название трека',  'Номер в рейтинге', 'Исполнитель', 'Альбом', 'Длительность'])
    df['Ненормативный контент'] = explicit
    df['Год вхождения в топ'] = year
    df['Ссылка на прослушивание'] = link
    df['Номер в рейтинге'] = df['Номер в рейтинге'].astype(int) 
    
    return df

Применям функцию к собранному списку из четырех ссылок-плейлистов. На выходе мы получаем четыре датафрейма, которые соединяем в один. 

In [99]:
dfs_list = []

for url in playlist_urls:
    wb.get(url)
    wb.implicitly_wait(10) 
    df = process_data(url)
    dfs_list.append(df)

wb.quit()
merged_df = pd.concat(dfs_list)
merged_df = merged_df.reset_index(drop = True) # сбрасываем индексацию, чтобы считало не до 100 каждый список, а общий до 400

In [100]:
merged_df

,Название трека,Номер в рейтинге,Исполнитель,Альбом,Длительность,Ненормативный контент,Год вхождения в топ,Ссылка на прослушивание
0,Последняя Любовь,1,MORGENSHTERN,Последняя Любовь - Single,2:42,1,2023,https://music.apple.com/ru/song/%D0%BF%D0%BE%D...
1,ПОДАРОК,2,АКУЛИЧ & Молодой Платон,ПОДАРОК - Single,2:39,1,2023,https://music.apple.com/ru/song/%D0%BF%D0%BE%D...
2,Царица,3,ANNA ASTI,Царица - Single,3:35,0,2023,https://music.apple.com/ru/song/%D1%86%D0%B0%D...
3,Спой,4,A.V.G & MACAN,Спой - Single,2:25,0,2023,https://music.apple.com/ru/song/%D1%81%D0%BF%D...
4,Можно Я С Тобой,5,AP$ENT,Можно Я С Тобой - Single,2:12,0,2023,https://music.apple.com/ru/song/%D0%BC%D0%BE%D...
...,...,...,...,...,...,...,...,...
395,Витаминка,96,Тима Белорусских,Твой первый диск - моя кассета,2:55,0,2020,https://music.apple.com/ru/song/%D0%B2%D0%B8%D...
396,ВЕЧЕРиНОЧКА,97,MONATIK & Вера Брежнева,V. - EP,3:58,0,2020,https://music.apple.com/ru/song/%D0%B2%D0%B5%D...
397,А уже фсё,98,Mitchel,А уже фсё - Single,2:14,0,2020,https://music.apple.com/ru/song/%D0%B0-%D1%83%...
398,Пчеловод,99,RASA,Пчеловод - Single,2:50,0,2020,https://music.apple.com/ru/song/%D0%BF%D1%87%D...


### Шаг 3.

Теперь перейдем к работе с сайтом [Genius](https://genius.com/). С него нам нужно собрать тексты и дату публикации трека в сеть. Так как сайт Genius работает особенно медленно в случаях, когда он контролируется сущностью веб-драйвер, мы сначала соберем ссылки на нужные нам страницы с текстами песен, а затем уже пройдемся по этим ссылкам и cоберем необходимые нам данные. Ссылки, по которым лежат тексты песен, мы тоже сохраним. <br> <br> Начнем с определения функции <font color='blue'>get_lyrics_links</font>, которая соберет для нас ссылки страницы с текстами. В этот раз поместим открытие и закрытие браузера внутрь функции.


In [35]:
list_songs = merged_df['Название трека'].to_list()
list_artists = merged_df['Исполнитель'].to_list()
lyrics_links = []

In [36]:
def get_lyrics_links(songlist, artistlist, url):
    
    '''Функция get_lyrics_links() принимает на вход список названий песен, список имен исполнителей и
    url-ссылку, затем производит поисковой запрос при помощи Selenium на основании элементов списка, 
    пробует забрать первую ссылку из выдачи по поисковому запросу в категории Songs и возвращает 
    list со списком ссылок, по которым лежат тексты песен.
    
    Функция создана для парсинга сайта https://genius.com/ и не предназначена для использования 
    на других ресурсах.
    
    Arguments:
        > songlist - list - список названий песен
        > artistlist - list - список имен исполнителей * длина списков совпадает
        > url - str - ccылка на платформу Genius;
    Returns:
        > lyrics_links - list - с ссылками на страницы, на которых лежат тексты песен. Элементы списка:
            : a - str - для страниц, для которых получилось собрать ссылку
            : 'No Data' - str - для страниц, для которых ссылку не получилось собрать
    Example:
        >>> example = get_lyrics_links(songlist = list_songs, artistlist = list_artists, 
                                       url = 'https://genius.com/')
        >>> example
    
    '''
    
    # 1. Запускаем веб-драйвер и даем ему ссылку на сайт
    # Задаем новые аргументы в options, которые должны поспособствовать более быстрй обработке сайта
    
    options = Options()
    options.add_argument("--disable-extensions") # отключить дополнения
    options.add_argument("start-maximized") 
    prefs = {"profile.default_content_settings.cookies": 2} # ограничение для кукис
    options.add_experimental_option("prefs", prefs)

    wb = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    wb.get(url)
    
    # Создаем пустой список
    
    lyrics_links = []
    
    # 2. Запускаем цикл и совершаем поисковой запрос, который каждый раз составляется из элементов списков.
    # Пробуем собрать ссылку, по которой лежит текст песни. 
    
    for i in range(len(songlist)):
        search = WebDriverWait(wb, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'input')))
        search.send_keys(songlist[i] + ' ' + artistlist[i] + '\n')
        wb.implicitly_wait(10)
    
        try:
            a = wb.find_elements(By.CLASS_NAME, 'mini_card')[1].get_attribute('href') 
            lyrics_links.append(a)
        except:
            lyrics_links.append('No Data')
            
    # 3. Завершаем работу браузера и возвращаем список из ссылок.
            
    wb.quit()
  
    return(lyrics_links)

Запустим нашу функцию и посмотрим на результат. 

In [37]:
lyrics_links = get_lyrics_links(list_songs, list_artists, url='https://genius.com/')

['https://genius.com/Morgenshtern-last-love-lyrics',
 'https://genius.com/Akyuliych-and-mp-gift-lyrics',
 'https://genius.com/Anna-asti-tsarina-lyrics',
 'https://genius.com/Macan-and-avg-sing-lyrics',
 'https://genius.com/Ap-ent-can-i-come-with-you-lyrics',
 'https://genius.com/Itsyk-tsyper-smoke-lyrics',
 'https://genius.com/Tdd-and-mona-farewell-lyrics',
 'https://genius.com/Almary-see-you-soon-lyrics',
 'https://genius.com/Veigel-goodbye-lyrics',
 'https://genius.com/Jakone-the-road-is-long-lyrics',
 'https://genius.com/Macan-and-navai-somewhere-in-depths-of-heart-lyrics',
 'https://genius.com/Macan-and-the-limba-maybe-lyrics',
 'https://genius.com/Basta-and-white-gallows-my-universe-lyrics',
 'https://genius.com/Aigel-glass-lyrics',
 'https://genius.com/Tatyana-kurtukova-mother-lyrics',
 'https://genius.com/Basta-at-dawn-lyrics',
 'https://genius.com/My-michelle-winter-in-the-heart-lyrics',
 'https://genius.com/Sqwore-lets-run-away-lyrics',
 'https://genius.com/Anna-asti-to-bars-l

In [39]:
lyrics_links # Получилось!

['https://genius.com/Morgenshtern-last-love-lyrics',
 'https://genius.com/Akyuliych-and-mp-gift-lyrics',
 'https://genius.com/Anna-asti-tsarina-lyrics',
 'https://genius.com/Macan-and-avg-sing-lyrics',
 'https://genius.com/Ap-ent-can-i-come-with-you-lyrics',
 'https://genius.com/Itsyk-tsyper-smoke-lyrics',
 'https://genius.com/Tdd-and-mona-farewell-lyrics',
 'https://genius.com/Almary-see-you-soon-lyrics',
 'https://genius.com/Veigel-goodbye-lyrics',
 'https://genius.com/Jakone-the-road-is-long-lyrics',
 'https://genius.com/Macan-and-navai-somewhere-in-depths-of-heart-lyrics',
 'https://genius.com/Macan-and-the-limba-maybe-lyrics',
 'https://genius.com/Basta-and-white-gallows-my-universe-lyrics',
 'https://genius.com/Aigel-glass-lyrics',
 'https://genius.com/Tatyana-kurtukova-mother-lyrics',
 'https://genius.com/Basta-at-dawn-lyrics',
 'https://genius.com/My-michelle-winter-in-the-heart-lyrics',
 'https://genius.com/Sqwore-lets-run-away-lyrics',
 'https://genius.com/Anna-asti-to-bars-l

Следующим шагом мы можем поочередно отправить через вебдрайвер запросы к этим страницам и собрать интересующую нас информацию (текст, дата публикации песни). <br> Определим функцию <font color='blue'>scrape_lyrics</font>, которая соберет для нас ссылки страницы с текстами, и применим ее к списку **lyrics_links**.

In [40]:
def scrape_lyrics(url):
    '''Функция scrape_lyrics() принимает на вход url-ссылку на индивидуальную страницу песни, cобирает данные 
    со страницы и возвращает два списка (list) с данными.
    
    Функция создана для парсинга сайта https://genius.com/ и не предназначена для использования 
    на других ресурсах.
    
    Arguments:
        > url - str - ccылка на страницу с текстом песни на платформе Genius;
    Returns:
        > lyrics - list - список с текстами песен;
        > date_ugly_list - list - список с датами публикации песен формата str
        
    Example:
        >>> example = scrape_lyrics(url = 'https://genius.com/Korol-i-shut-sorcerers-doll-lyrics')
        >>> example
    
    '''
    
    # 1. Передаем в веб-драйвер ссылку
    
    wb.get(url)
    
    # 2. Собираем элементы, в которых хранится текст песни (в завимисоти от длины их один или два)
    
    try:
        lyrics_a_lot = [wb.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div[3]/div/div/div[2]'), wb.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div[3]/div/div/div[5]')]
        one_song_lyrics = [i.text for i in lyrics_a_lot]
        one_song_lyrics = one_song_lyrics[0] + one_song_lyrics[1]
    except:
        one_song_lyrics = wb.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div[3]/div/div/div[2]')
        
    lyrics.append(one_song_lyrics)
    
    # 3. Собираем элементы, в которых хранится дата выхода песни
    
    try:
        date_ugly = WebDriverWait(wb, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/main/div[1]/div[3]/div[1]/div[2]/div/span[1]/span'))).text
        date_ugly_list.append(date_ugly)
    except:
        date_ugly_list.append('No Data')
        
    return(lyrics, date_ugly_list)

In [41]:
# создаем списки

lyrics = []
date_ugly_list = []

# запускаем веб-драйвер

options = Options()
options.add_argument("--disable-extensions")
options.add_argument("start-maximized") 
prefs = {"profile.default_content_settings.cookies": 2}
options.add_experimental_option("prefs", prefs)

wb = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
for url in lyrics_links:
    try:
        scrape_lyrics(url)
    except:
        lyrics.append('No Data')
        date_ugly_list.append('No Data')
        
wb.quit()

### Шаг 4. Объединение датафреймов и немного обработки данных.

Проверим, что все получилось собрать. <br>  Пока данные представлены в виде списков и с ними леге взаимодействовать, отформатируем их: почистим текст от лишних символов и приведем дату к формату *datetime*. <br> После этого добавим новые столбцы к существующему датафрейму.

In [42]:
lyrics

['[Текст песни «Последняя Любовь»]\n\n[Куплет 1: MORGENSHTERN]\nТы снова куришь, снова слёзы на твоих щеках\nОпять ждала меня бухого до семи утра (Ага)\nПообещал, что я всю жизнь буду любить тебя\nЯ рэпер, зай. Мои слова не стоят ни хуя (Ага)\nИ я подонок, я изменщик, я газлайтер и абьюзер\nЯ не нравлюсь твоей маме, да и хуй с ней (Ну, допустим)\nДетка, хватит мне уже давать последний шанс (Ага)\nСчастье — это не для нас\n\n[Припев: MORGENSHTERN & Lida]\nПоследняя любовь, пепел на простынях\nБери все мои вещи, выкидывай из окна\nПоследняя любовь. Плачешь, а мне плевать\nТы больше не поверишь никому и никогда\n\n[Куплет 2: MORGENSHTERN]\nА ты вернула все подарки, отписалась в Инсте\nВ общем, сделала всю ту хуйню, что делают все\nИ я оставлю тебе травмы, когда брошу тебя\nВедь ты же так искала парня, что похож на отца\nИ когда будешь ныть подругам то, что я уебан\nПопроси, чтоб они все мне перестали писать\nЯ с детства знал, что все девчонки любят плохих парней\nНо, детка, я не плохой па

In [59]:
# замечаю, что в некоторых случаях код вернул веб-элементы в lyrics -- меняем их на 'No Data'
lyrics_ = []

for i in lyrics:
    lyrics_.append((lambda x: 'No Data' if type(x) != str else x)(i))

# потом разберемся

In [60]:
len(lyrics_)

400

In [43]:
date_ugly_list

['Mar. 8, 2024',
 'Feb. 16, 2024',
 'Jul. 14, 2023',
 'Jan. 26, 2024',
 'Jul. 7, 2022',
 'Aug. 11, 2023',
 'Oct. 20, 2023',
 'Feb. 16, 2024',
 'Feb. 9, 2024',
 'Feb. 23, 2024',
 'Mar. 8, 2024',
 'Mar. 1, 2024',
 'Mar. 8, 2024',
 'Nov. 20, 2020',
 'Nov. 4, 2022',
 'Oct. 1, 2019',
 'Jul. 30, 2021',
 'Nov. 26, 2021',
 'Jun. 3, 2022',
 'Dec. 14, 2018',
 'Mar. 31, 2023',
 'Aug. 5, 2022',
 'Feb. 22, 2024',
 'Apr. 2, 2023',
 'No date',
 'Jan. 12, 2024',
 'Oct. 27, 2023',
 'Feb. 10, 2024',
 'Mar. 1, 2024',
 'Mar. 28, 2022',
 'Jun. 23, 2023',
 'Sep. 9, 2016',
 'Mar. 8, 2024',
 'Dec. 16, 2022',
 'Jan. 12, 2024',
 'May. 24, 2017',
 'Nov. 28, 2023',
 'Mar. 8, 2024',
 'Nov. 23, 2015',
 'Oct. 4, 2022',
 'Jan. 1, 1989',
 'Dec. 16, 2017',
 'Feb. 14, 2019',
 'Jun. 24, 2022',
 'Aug. 13, 2021',
 'Oct. 13, 2023',
 'Feb. 9, 2018',
 'Jul. 17, 2023',
 'No date',
 'Dec. 8, 2023',
 'No date',
 'Nov. 25, 2022',
 'Apr. 28, 2023',
 'Oct. 7, 2022',
 'Feb. 16, 2024',
 'May. 12, 2023',
 'Apr. 29, 2022',
 'Feb. 7, 20

In [45]:
print(len(date_ugly_list), len(lyrics_), len(lyrics_links))

400 400 400


In [61]:
# чистим текст от совсем не нужных символов

for i in range(len(lyrics_)):
    lyrics_[i] = re.sub('["\u2005", "\u205f", "\n"]', ' ', lyrics_[i])
    lyrics_[i] = re.sub(r'\[.*?\]', '', lyrics_[i]).strip()
    
lyrics_

['Ты снова куришь  снова слёзы на твоих щеках Опять ждала меня бухого до семи утра (Ага) Пообещал  что я всю жизнь буду любить тебя Я рэпер  зай. Мои слова не стоят ни хуя (Ага) И я подонок  я изменщик  я газлайтер и абьюзер Я не нравлюсь твоей маме  да и хуй с ней (Ну  допустим) Детка  хватит мне уже давать последний шанс (Ага) Счастье — это не для нас   Последняя любовь  пепел на простынях Бери все мои вещи  выкидывай из окна Последняя любовь. Плачешь  а мне плевать Ты больше не поверишь никому и никогда   А ты вернула все подарки  отписалась в Инсте В общем  сделала всю ту хуйню  что делают все И я оставлю тебе травмы  когда брошу тебя Ведь ты же так искала парня  что похож на отца И когда будешь ныть подругам то  что я уебан Попроси  чтоб они все мне перестали писать Я с детства знал  что все девчонки любят плохих парней Но  детка  я не плохой парень  я плохой человек И я нормальным быть пытался Но я заебался Я рождён уёбком  не достойным любви Извини   Последняя любовь  пепел на п

In [62]:
# создаем маленькую функцию, которая преобразует дату в объект datetime

def datemaker(date_ugly):
    '''Функция datemaker() принимает на вход cписок дат в формате "Мес. день год" 
    и превращает его в список с объектами формата datetime
    
    Arguments:
        > date_ugly - list - список дат в формате для преобразования
    Returns:
        > dt_object_list - list - список с объектами формата datetime
        
    Example:
        >>> example = datemaker(['Mar. 20, 2019', 'Dec. 6, 2019', 'Jan. 31, 2020'])
        >>> example
    
    '''
    dt_object_list = []
    
    for i in date_ugly:
        try: 
            i = re.sub('[,.]', '', i)
            dt_object = datetime.strptime(i, '%b %d %Y')
            dt_object_list.append(dt_object)
        except:
            dt_object_list.append('No Data')
            

    return dt_object_list

In [101]:
merged_df['Текст песни'] = lyrics_ # добавили текст
merged_df['Дата выхода'] = datemaker(date_ugly_list) # добавили дату
merged_df['Ссылка на текст'] = lyrics_links # добавили ссылку

In [102]:
merged_df

,Название трека,Номер в рейтинге,Исполнитель,Альбом,Длительность,Ненормативный контент,Год вхождения в топ,Ссылка на прослушивание,Текст песни,Дата выхода,Ссылка на текст
0,Последняя Любовь,1,MORGENSHTERN,Последняя Любовь - Single,2:42,1,2023,https://music.apple.com/ru/song/%D0%BF%D0%BE%D...,Ты снова куришь снова слёзы на твоих щеках Оп...,2024-03-08 00:00:00,https://genius.com/Morgenshtern-last-love-lyrics
1,ПОДАРОК,2,АКУЛИЧ & Молодой Платон,ПОДАРОК - Single,2:39,1,2023,https://music.apple.com/ru/song/%D0%BF%D0%BE%D...,Захожу на праздник: what the fuck? Диджей полн...,2024-02-16 00:00:00,https://genius.com/Akyuliych-and-mp-gift-lyrics
2,Царица,3,ANNA ASTI,Царица - Single,3:35,0,2023,https://music.apple.com/ru/song/%D1%86%D0%B0%D...,Все твои романы — тяжёлый вид спорта Каждый бы...,2023-07-14 00:00:00,https://genius.com/Anna-asti-tsarina-lyrics
3,Спой,4,A.V.G & MACAN,Спой - Single,2:25,0,2023,https://music.apple.com/ru/song/%D1%81%D0%BF%D...,У я Солнце в заход басота в расход Чёрный Ra...,2024-01-26 00:00:00,https://genius.com/Macan-and-avg-sing-lyrics
4,Можно Я С Тобой,5,AP$ENT,Можно Я С Тобой - Single,2:12,0,2023,https://music.apple.com/ru/song/%D0%BC%D0%BE%D...,Горят фонари Я бы до зари с тобой мог говорить...,2022-07-07 00:00:00,https://genius.com/Ap-ent-can-i-come-with-you-...
...,...,...,...,...,...,...,...,...,...,...,...
395,Витаминка,96,Тима Белорусских,Твой первый диск - моя кассета,2:55,0,2020,https://music.apple.com/ru/song/%D0%B2%D0%B8%D...,Так беззаботно уходит вдаль наша с тобой жизнь...,2019-01-29 00:00:00,https://genius.com/Tima-belorusskih-vitamin-ly...
396,ВЕЧЕРиНОЧКА,97,MONATIK & Вера Брежнева,V. - EP,3:58,0,2020,https://music.apple.com/ru/song/%D0%B2%D0%B5%D...,Тихие слова линии твоих губ Кругом голова са...,2020-06-11 00:00:00,https://genius.com/Monatik-party-lyrics
397,А уже фсё,98,Mitchel,А уже фсё - Single,2:14,0,2020,https://music.apple.com/ru/song/%D0%B0-%D1%83%...,А уже всё всё всё всё А надо было раньше Говор...,2020-07-07 00:00:00,https://genius.com/Mitchel-and-and-its-over-ly...
398,Пчеловод,99,RASA,Пчеловод - Single,2:50,0,2020,https://music.apple.com/ru/song/%D0%BF%D1%87%D...,Ты пчела — я пчеловод А мы любим мёд А мне пов...,2019-07-02 00:00:00,https://genius.com/Rasa-beekeeper-lyrics


***

## Часть 2. Обработка данных

### Шаг 1. Пропуски

В первую очередь разберемся с пропусками в данных: посмотрим, сколько их и где они сосредоточены. При этом нас будут интересовать те переменные, которые были собраны с [Genius](https://genius.com/), в особенности текстовые данные, которые пригодятся для демонстрации  базового текстового анализа. Создадим датасет, который будет состоять только из строк с пропущенными переменными.

In [103]:
df_na = merged_df[(merged_df['Дата выхода']=='No Data') | (merged_df['Текст песни']=='No Data') | (merged_df['Ссылка на текст']=='No Data')]

In [104]:
df_na

,Название трека,Номер в рейтинге,Исполнитель,Альбом,Длительность,Ненормативный контент,Год вхождения в топ,Ссылка на прослушивание,Текст песни,Дата выхода,Ссылка на текст
15,На заре,16,Баста,На заре - Single,5:06,0,2023,https://music.apple.com/ru/song/%D0%BD%D0%B0-%...,No Data,2019-10-01 00:00:00,https://genius.com/Basta-at-dawn-lyrics
21,Amathole,22,Joezi & Lizwi,Amathole - Single,8:07,0,2023,https://music.apple.com/ru/song/amathole/16408...,No Data,2022-08-05 00:00:00,https://genius.com/Joezi-and-lizwi-amathole-ly...
24,Харизма,25,Wallem,Харизма - Single,2:16,0,2023,https://music.apple.com/ru/song/%D1%85%D0%B0%D...,Not available,No Data,Ссылка на текст недоступна
48,Шатенка,49,TIGO,Шатенка - Single,2:28,0,2023,https://music.apple.com/ru/song/%D1%88%D0%B0%D...,Not available,No Data,Ссылка на текст недоступна
50,"Я М5, ты AMG",51,VEIGEL,"Я М5, ты AMG - Single",2:19,0,2023,https://music.apple.com/ru/song/%D1%8F-%D0%BC5...,Not available,No Data,Ссылка на текст недоступна
67,Кукла колдуна,68,Король и Шут,Акустический альбом,3:23,0,2023,https://music.apple.com/ru/song/%D0%BA%D1%83%D...,Тёмный мрачный коридор Я на цыпочках как вор...,No Data,https://genius.com/Korol-i-shut-sorcerers-doll...
131,Пополам,32,BRANYA & MACAN,Пополам - Single,2:21,0,2022,https://music.apple.com/ru/song/%D0%BF%D0%BE%D...,No Data,No Data,https://genius.com/Branya-and-macan-demo-lyrics
146,На сиреневой луне (Леонид Агутин Cover),47,JONY,На сиреневой луне (Леонид Агутин Cover),2:07,0,2022,https://music.apple.com/ru/song/%D0%BD%D0%B0-%...,Не застилая горизонт твоей луны которой нет От...,No Data,https://genius.com/Leonid-agutin-on-the-purple...
217,Пополам,18,BRANYA & MACAN,Пополам - Single,2:21,0,2021,https://music.apple.com/ru/song/%D0%BF%D0%BE%D...,No Data,No Data,https://genius.com/Branya-and-macan-demo-lyrics
223,По щекам слёзы,24,КУЧЕР & JANAGA,По щекам слёзы - Single,2:59,0,2021,https://music.apple.com/ru/song/%D0%BF%D0%BE-%...,Not available,No Data,Ссылка на текст недоступна


Мы видим, что, действительно, в датасете есть пропущенные данные, в том числе и в столбце *'Текст песни'*. Так как наша выборка весьма небольшая, хотелось бы приблизить к минимуму количество текстов, которые окажутся не учтены в финальном анализе. <br><br> Методом ручного тыка выяснилось, что не для всех песен текстов действительно не существует в базе [Genius](https://genius.com/) -- некоторые из них Selenium по тем или иным причинам просто пропустил (вероятно, из-за медленной работы браузера). <br><br> Поэтому попробуем запустить сбор текстов для этих элементов еще раз, немного скорректировав функцию, чтобы увеличить шанс того, что нужный текст дейсвительно нам выпадет: например, попробуем перетасовать местами название трека и имя исполнителя и обращаться только по имени первого исполнителя в случае, если исполнителей несколько. 

In [88]:
def resolve_mistakes(songlist, artistlist, merged_df, df_na):
    
    '''Функция resolve_mistakes() принимает на вход список названий песен, список имен исполнителей, большой датасет
    и датасет с данными, в которых есть пропуски.
    При помощи Selenium функция отправляет повторный поисковой запрос и пробует забрать первую ссылку в категории Songs, 
    затем открывает эту ссылку и забирает текст песни, заполняя пропуски в датасете.
    Функция возвращает pandas.DataFrame с заполненными пропусками.
    
    Функция создана для парсинга сайта https://genius.com/ и не предназначена для использования 
    на других ресурсах.
    
    Arguments:
        > songlist - list - список названий песен;
        > artistlist - list - список имен исполнителей;
        > merged_df - pandas.DataFrame - искомый датасет с пропусками;
        > df_na - pandas.DataFrame - датасет с данными, в которых есть пропуски (получен путем фильтрации merged_df)
        
    Returns:
        > merged_df - pandas.DataFrame - искомый датасет без пропусков
    Example:
        >>> example = get_lyrics_links(songlist = list_songs, artistlist = list_artists, 
                                       url = 'https://music.apple.com/ru/browse')
        >>> example = merged_df
    
    '''
    
    # 1. Запускаем цикл и совершаем поисковой запрос, который каждый раз составляется из элементов списков.
    
    for one, row in merged_df.iterrows():
        for two, rows in df_na.iterrows():
            if one == two:
                search = WebDriverWait(wb, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'input')))
                search.send_keys(artistlist[one].split()[0] + ' ' + songlist[one] + '\n')
                wb.implicitly_wait(10)
                
    # 2. Пробуем собрать ссылку, по которой лежит текст песни. 
    
                try:
                    a = wb.find_elements(By.CLASS_NAME, 'mini_card')[1].get_attribute('href') 
                    merged_df.at[one, 'Ссылка на текст'] = a
                    wb.get(a)
                    
    # 3. Пробуем собрать текст песни. 
                                
                    try:
                        lyrics_a_lot = [wb.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div[3]/div/div/div[2]'), wb.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div[3]/div/div/div[5]')]
                        one_song_lyrics = [i.text for i in lyrics_a_lot]
                        one_song_lyrics = one_song_lyrics[0] + one_song_lyrics[1]
                        one_song_lyrics = re.sub('["\u2005", "\u205f", \n"]', ' ', one_song_lyrics)
                        one_song_lyrics = re.sub(r'\[.*?\]', '', one_song_lyrics).strip()
                        merged_df.at[one, 'Текст песни'] = one_song_lyrics
                    except:
                        try: 
                            one_song_lyrics = wb.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div[3]/div/div/div[2]').text
                            one_song_lyrics = re.sub('["\u2005", "\u205f", \n"]', ' ', one_song_lyrics)
                            one_song_lyrics = re.sub(r'\[.*?\]', '', one_song_lyrics).strip()
                            merged_df.at[one, 'Текст песни'] = one_song_lyrics
                        except:
                            merged_df.at[one, 'Текст песни'] = 'No Data'
                            
                    wb.back() # зацикливаем действие
                    
    # 3. Если ничего не получилось -- поставляем значение No Data
                        
                except:
                    merged_df.at[one, 'Текст песни'] = 'No Data'
                    merged_df.at[one, 'Ссылка на текст'] = 'No Data'
  
    return(merged_df)

In [89]:
options = Options()
options.add_argument("--disable-extensions") # отключить дополнения
options.add_argument("start-maximized") 
prefs = {"profile.default_content_settings.cookies": 2} # ограничение для кукис
options.add_experimental_option("prefs", prefs)

wb = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
wb.get(url = 'https://genius.com/')

resolve_mistakes(list_songs, list_artists, merged_df, df_na)

wb.quit()

In [90]:
df_na2 = merged_df[(merged_df['Дата выхода']=='No Data') | (merged_df['Текст песни']=='No Data') | (merged_df['Ссылка на текст']=='No Data')]
df_na2

,Номер в рейтинге,Название трека,Исполнитель,Дата выхода,Год вхождения в топ,Длительность,Ненормативный контент,Сингл-трек,Текст песни,Ссылка на прослушивание,Ссылка на текст
24,25,Харизма,Wallem,No Data,2023,136,0,1,No Data,https://music.apple.com/ru/song/%D1%85%D0%B0%D...,No Data
48,49,Шатенка,TIGO,No Data,2023,148,0,1,No Data,https://music.apple.com/ru/song/%D1%88%D0%B0%D...,No Data
50,51,"Я М5, ты AMG",VEIGEL,No Data,2023,139,0,1,No Data,https://music.apple.com/ru/song/%D1%8F-%D0%BC5...,No Data
67,68,Кукла колдуна,Король и Шут,No Data,2023,203,0,0,Тёмный мрачный коридор Я на цыпочках как вор...,https://music.apple.com/ru/song/%D0%BA%D1%83%D...,https://genius.com/Korol-i-shut-sorcerers-doll...
131,32,Пополам,BRANYA & MACAN,No Data,2022,141,0,1,Да я видел не мало но нашёл тебя по глазам ...,https://music.apple.com/ru/song/%D0%BF%D0%BE%D...,https://genius.com/Branya-and-macan-in-half-ly...
146,47,На сиреневой луне (Леонид Агутин Cover),JONY,No Data,2022,127,0,0,Ты забудешь обо мне на сиреневой луне\nМожет т...,https://music.apple.com/ru/song/%D0%BD%D0%B0-%...,https://genius.com/Jony-na-sirenevoy-lune-lyrics
217,18,Пополам,BRANYA & MACAN,No Data,2021,141,0,1,Да я видел не мало но нашёл тебя по глазам ...,https://music.apple.com/ru/song/%D0%BF%D0%BE%D...,https://genius.com/Branya-and-macan-in-half-ly...
223,24,По щекам слёзы,КУЧЕР & JANAGA,No Data,2021,179,0,1,Поменял я новый стиль Но стал немного скован А...,https://music.apple.com/ru/song/%D0%BF%D0%BE-%...,https://genius.com/Meddved-blop-blop-lyrics
225,26,El Problema,Morgenstern & Roni Schwartz & SHINGA,No Data,2021,137,0,1,E-E-El Primero with big dick I have a problem:...,https://music.apple.com/ru/song/el-problema/16...,https://genius.com/Genius-english-translations...
298,99,ZITTI E BUONI,Måneskin,No Data,2021,192,1,1,They don't know what I'm talking about Clothes...,https://music.apple.com/ru/song/zitti-e-buoni/...,https://genius.com/Genius-english-translations...


Выглядит уже лучше! Конечно, можно было бы заняться заполнением всех пропусков, но в силу ограниченности ресурсов остановимся на этом. 

### Шаг 2. Форматирование данных

В продолжении мы можем также обработать некоторые другие столбцы датафрейма, чтобы в следующей части вывести по ним наиболее содержательные описательные статистики:
1. Для столбца "*Альбом*" создадим дамми-переменную (1-0), которая будет указывать, входит ли трек в альбом или является синглом.
2. Значения в столбце "*Длительность*" представим в формате int, переведя запись формата {мм:сс} в секунды.

In [105]:
merged_df['Сингл-трек'] = merged_df['Альбом'].str.contains('Single').astype(int)
merged_df.drop(columns = ['Альбом'], inplace = True)

In [106]:
def timemaker(lenght_list):
    '''Функция timemaker() принимает на вход cписок из строк, обозначающих длительность трека в формате "мм:сс",
    и превращает его в список с числовыми значениями, обозначающими длительность трека в секундах.
    
    Arguments:
        > lenght_list - list - cписок из строк в формате "мм:сс"
    Returns:
        > length_format - list - список с числовыми значениями (длительность трека в секундах)
        
    Example:
        >>> example = datemaker(['2:42', '2:39', '2:35'])
        >>> example
    '''
    
    length_format = []
    for i in lenght_list:
        try:
            i = i.split(':')
            i = int(int(i[0])*60 + int(i[1]))
            length_format.append(i)
        except:
            length_format.append('No Data')

    return length_format

In [107]:
lenght_list = merged_df['Длительность'].to_list()
length_formated = timemaker(lenght_list)
merged_df['Длительность'] = length_formated

### Шаг 3. Сохранение итогового файла

Наконец, переставим столбцы местами в датафрейме, чтобы в нем было легче ориентироваться.

In [108]:
merged_df = merged_df[['Номер в рейтинге', 'Название трека', 'Исполнитель', 
                       'Дата выхода', 'Год вхождения в топ', 'Длительность', 
                       'Ненормативный контент', 'Сингл-трек', 'Текст песни', 
                       'Ссылка на прослушивание', 'Ссылка на текст']]
merged_df

,Номер в рейтинге,Название трека,Исполнитель,Дата выхода,Год вхождения в топ,Длительность,Ненормативный контент,Сингл-трек,Текст песни,Ссылка на прослушивание,Ссылка на текст
0,1,Последняя Любовь,MORGENSHTERN,2024-03-08 00:00:00,2023,162,1,1,Ты снова куришь снова слёзы на твоих щеках Оп...,https://music.apple.com/ru/song/%D0%BF%D0%BE%D...,https://genius.com/Morgenshtern-last-love-lyrics
1,2,ПОДАРОК,АКУЛИЧ & Молодой Платон,2024-02-16 00:00:00,2023,159,1,1,Захожу на праздник: what the fuck? Диджей полн...,https://music.apple.com/ru/song/%D0%BF%D0%BE%D...,https://genius.com/Akyuliych-and-mp-gift-lyrics
2,3,Царица,ANNA ASTI,2023-07-14 00:00:00,2023,215,0,1,Все твои романы — тяжёлый вид спорта Каждый бы...,https://music.apple.com/ru/song/%D1%86%D0%B0%D...,https://genius.com/Anna-asti-tsarina-lyrics
3,4,Спой,A.V.G & MACAN,2024-01-26 00:00:00,2023,145,0,1,У я Солнце в заход басота в расход Чёрный Ra...,https://music.apple.com/ru/song/%D1%81%D0%BF%D...,https://genius.com/Macan-and-avg-sing-lyrics
4,5,Можно Я С Тобой,AP$ENT,2022-07-07 00:00:00,2023,132,0,1,Горят фонари Я бы до зари с тобой мог говорить...,https://music.apple.com/ru/song/%D0%BC%D0%BE%D...,https://genius.com/Ap-ent-can-i-come-with-you-...
...,...,...,...,...,...,...,...,...,...,...,...
395,96,Витаминка,Тима Белорусских,2019-01-29 00:00:00,2020,175,0,0,Так беззаботно уходит вдаль наша с тобой жизнь...,https://music.apple.com/ru/song/%D0%B2%D0%B8%D...,https://genius.com/Tima-belorusskih-vitamin-ly...
396,97,ВЕЧЕРиНОЧКА,MONATIK & Вера Брежнева,2020-06-11 00:00:00,2020,238,0,0,Тихие слова линии твоих губ Кругом голова са...,https://music.apple.com/ru/song/%D0%B2%D0%B5%D...,https://genius.com/Monatik-party-lyrics
397,98,А уже фсё,Mitchel,2020-07-07 00:00:00,2020,134,0,1,А уже всё всё всё всё А надо было раньше Говор...,https://music.apple.com/ru/song/%D0%B0-%D1%83%...,https://genius.com/Mitchel-and-and-its-over-ly...
398,99,Пчеловод,RASA,2019-07-02 00:00:00,2020,170,0,1,Ты пчела — я пчеловод А мы любим мёд А мне пов...,https://music.apple.com/ru/song/%D0%BF%D1%87%D...,https://genius.com/Rasa-beekeeper-lyrics


Последним шагом сохраним наши датафреймы, чтобы продолжить работать с ними в другом ноутбуке.

In [109]:
merged_df.to_excel('ПРОЕКТ_собранные_Данные.xlsx')

In [1]:
merged_df.to_csv('ПРОЕКТ_собранные_Данные.csv', sep='\t', encoding='utf-8')

NameError: name 'merged_df' is not defined

### Продолжение анализа датафрейма в файле "ПРОЕКТ_Анализ_данных_Apple+Genius.ipynb"